In [1]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = "/Users/duhameleve/Desktop/M2_SNS/M2SNS/Sante/Github/HAH913E-2024-projet/AX3 Project/Sans titre/data/NoneSmokerCardio.csv"

data = pd.read_csv(file_path)

timestamp_column = data.columns[0]
signal_columns = data.columns[1:]   # All remaining columns are sensor data

# Convert the timestamp column to datetime
data[timestamp_column] = pd.to_datetime(data[timestamp_column], errors='coerce')  # Handle invalid dates

# Calculate the sampling interval (in seconds)
sampling_interval = (
    pd.to_datetime(data[timestamp_column].iloc[1]) -  # Access a single row
    pd.to_datetime(data[timestamp_column].iloc[0])
).total_seconds()

# Create a relative time axis (in minutes)
data['relative_time_minutes'] = data.index * sampling_interval

# Define axis labels for clarity
axis_labels = ['X-axis', 'Y-axis', 'Z-axis']
colors = {'X': 'blue', 'Y': 'red', 'Z': 'lightgreen'}  # Define colors for each signal

In [2]:
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

#Sélectionner l'axe avec la plus grande variance
signal_columns = data.columns[1:-1]  # Exclure le timestamp et le temps relatif
variances = data[signal_columns].var()
selected_axis = variances.idxmax()

# Définir la fréquence d'échantillonnage et de coupure
fs = 1 / sampling_interval  # Fréquence d'échantillonnage en Hz
cutoff = 2  # Fréquence de coupure en Hz (ajustez si nécessaire)

# Appliquer le filtre passe-bas
data[selected_axis] = lowpass_filter(data[selected_axis], cutoff, fs)

# Détecter les pics dans le signal filtré
peaks, _ = find_peaks(data[selected_axis], height=0.25, distance=fs * 0.1)  # Ajuster les paramètres si nécessaire

# Calculer les temps associés aux pics détectés
peak_times = data['relative_time_minutes'].iloc[peaks]

# Calculer les intervalles entre les pics (en secondes)
intervals = peak_times.diff().dropna()

# Calculer la cadence (pas par seconde)
cadence_per_second = 1 / intervals

# Convertir la cadence en pas par minute
cadence_per_minute = cadence_per_second * 60

# Afficher les statistiques
print(f"Nombre total de pics détectés : {len(peaks)}")
print(f"Intervalle moyen entre les pics : {intervals.mean():.2f} secondes")
print(f"Cadence moyenne : {cadence_per_minute.mean():.2f} pas par minute")

# Visualiser le signal filtré et les pics détectés
plt.figure(figsize=(12, 6))
plt.plot(data['relative_time_minutes'], data[selected_axis], label=f"Signal filtré {selected_axis}")
plt.plot(peak_times, data[selected_axis].iloc[peaks], "x", label="Pics détectés", color='red')
plt.title("Détection des Pics dans le Signal Filtré")
plt.xlabel("Temps (minutes)")
plt.ylabel("Accélération (g)")
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'lowpass_filter' is not defined